In [3]:
! pip install kaggle
from google.colab import drive
drive.mount('/content/drive')
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip features.csv.zip
!unzip train.csv.zip
!unzip test.csv.zip
!unzip sampleSubmission.csv.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 775MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  features.csv.zip
  inflating: features.csv            
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  sampleSubmission.csv.zip
  inflating: sampleSubmission.csv    


In [2]:
!pip install -q numpy scikit-learn neuralforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
import pandas as pd
from neuralforecast.models import NBEATS
from neuralforecast import NeuralForecast
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
# import wandb
# run = wandb.init()
# artifact = run.use_artifact('tchan22-free-university-of-tbilisi-/walmart-forecasting/pipeline_artifact:v0', type='model')


In [6]:
import pandas as pd
test = pd.read_csv("test.csv")
test

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


# Classes needed for model to work

In [7]:
class N_Beats(NBEATS):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.9)

    def set_optim(self, optimizer):
        self.optimizer = optimizer

    def set_scheduler(self, scheduler):
        self.scheduler = scheduler

    def configure_optimizers(self):
        return {
            'optimizer': self.optimizer,
            'lr_scheduler': {
                'scheduler': self.scheduler,
                'interval': 'epoch',
                'frequency': 1
            }
        }

class Preprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        x_copy = X.copy()
        if 'y' not in x_copy.columns:
            x_copy['y'] = 0
        x_copy['Date'] = pd.to_datetime(x_copy['Date'])
        return x_copy

class NBEATSModel(BaseEstimator, TransformerMixin):
    def __init__(self, models: list[NBEATS] = None):
        if models is None or len(models) == 0:
            device = 'gpu' if torch.cuda.is_available() else 'cpu'
            models = [N_Beats(
                input_size=40,
                h=48,
                max_steps=1300,
                batch_size=64,
                stack_types=['identity', 'trend', 'seasonality'],
                n_blocks=[1, 1, 1],
                random_seed=42,
                accelerator=device,
                devices=1,
                logger=True,
                enable_progress_bar=False,
                enable_model_summary=False,
            )]
        self.nf = NeuralForecast(models=models, freq='W-FRI')

    def fit(self, X, y=None):
        df = self.create_nbeats(X, y)
        df.sort_values(by=['ds'], inplace=True)
        self.nf.fit(df)
        return self

    def transform(self, X):
        preds = self.nf.predict()
        real = self.create_nbeats(X, X['y'])
        merged = pd.merge(real, preds, on=['unique_id', 'ds'], how='left')
        merged.fillna(0, inplace=True)
        return merged

    def predict(self, X=None):
        return self.transform(X)

    def create_nbeats(self, x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:
        df = pd.DataFrame()
        df["ds"] = pd.to_datetime(x["Date"])
        df["Store"] = x["Store"]
        df["Dept"] = x["Dept"]
        df["IsHoliday"] = x["IsHoliday"].astype(int)
        df["unique_id"] = df["Dept"].astype(str) + "_" + df["Store"].astype(str)
        df["y"] = y.values
        return df[["unique_id", "ds", "y", "IsHoliday"]].copy()

In [8]:
import joblib
model_path = f"pipeline.pkl"
loaded_model = joblib.load(model_path)

In [9]:
predictions = loaded_model.predict(test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [12]:
submission = pd.read_csv("sampleSubmission.csv")
submission

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0
...,...,...
115059,45_98_2013-06-28,0
115060,45_98_2013-07-05,0
115061,45_98_2013-07-12,0
115062,45_98_2013-07-19,0


In [ ]:
predictions

In [ ]:
submission_df = pd.DataFrame()
submission_df['Id'] = predictions['unique_id'] + '_' + predictions['ds'].astype(str)
submission_df['Weekly_Sales'] = predictions['N_Beats']

In [16]:
submission.to_csv("submission_nbeats.csv", index=False)